In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os
from numpy.typing import NDArray
from PIL import Image

# TODO: for This Dataset, 250k for jpeg is too much. I tried and pictures deviate a lot from desired size.

In [80]:
def get_image(file_path:Path)-> NDArray|None:
    if os.path.isfile(file_path) and file_name.lower().endswith(".png"):
        image = Image.open(file_path)
        if image.mode == "RGBA":
            image = image.convert("RGB")
            # array = np.array(image)
        return image
    print("returned none")    
    return None   

def save_jpeg_img(img:Image,name:str,quality: int) -> None:
    img.save(os.path.join(destinationFolder, f"{name}.jpeg"),format="JPEG",quality=quality)
    
def get_image_size(folder:Path,base_name:str,format:str) -> int:
    # test_image_path = os.path.join(destinationFolder, f"{base_name}.{format}")
    test_image_path = os.path.join(folder, f"{base_name}.{format}")
    test_image_size = os.path.getsize(test_image_path) // 1000
    return int(test_image_size)
def remove_image(folder:Path,base_name:str,format:str):
    file_path = os.path.join(folder, f"{base_name}.{format}")
    os.remove(file_path)

In [81]:
sourceFolder = "cropped_images"
destinationFolder = "compressed_draft"

if not os.path.exists(destinationFolder):
    os.makedirs(destinationFolder)


In [82]:
pillowFormats = [("JPEG", "jpeg"), ("JPEG2000", "jp2"), ("WEBP", "webp")]
imagecodecsFormats = [
    ("jpegxl_encode", "jxl"),
    ("avif_encode", "avif"),]
sorted_sourcefolder_walk = sorted(os.listdir(sourceFolder), key=lambda x: int(x[:-4]))


In [83]:
desired_jpeg_size_kb = 35


tolerated_file_size_deviation = 2
base_jpeg_quality= 60
jpeg_quality_step = 2

jpeg_quality_70k_compression : dict[str:int] = {}
jpeg_max_quality = 95
jpeg_min_quality = 0



for file_name in sorted_sourcefolder_walk:
    file_path = os.path.join(sourceFolder, file_name)
    image= get_image(file_path)
    base_name = os.path.splitext(file_name)[0]
    png_size = get_image_size(sourceFolder,base_name,'png')
    "Some pictures are too small, meaning even with highest compression ratio they do not reach our desired size on disk"
    if png_size < 1.5* desired_jpeg_size_kb:
        continue
    "First attempt to compress to desired file size."
    save_jpeg_img(img=image,name=base_name,quality=base_jpeg_quality)
    jpeg_size = get_image_size(destinationFolder,base_name,'jpeg')
    file_size_one_step_before = 0
    reached_jpeg_quality = False
    new_jpeg_quality = base_jpeg_quality
    while not reached_jpeg_quality:
        
        if np.abs(jpeg_size-desired_jpeg_size_kb) < tolerated_file_size_deviation:
            jpeg_quality_70k_compression[base_name] = new_jpeg_quality
            image.close()
            print(f"{base_name} was below threshold")
            break
            
        elif jpeg_size < desired_jpeg_size_kb:
            """we should increase jpeg quality to reach desired file size on disk. """
            new_jpeg_quality += jpeg_quality_step
            save_jpeg_img(img=image,name=base_name,quality=new_jpeg_quality)
            jpeg_size = get_image_size(destinationFolder,base_name,'jpeg')
            
            if np.abs(file_size_one_step_before - desired_jpeg_size_kb) <= np.abs(jpeg_size-desired_jpeg_size_kb):
                
                if not np.abs(jpeg_size-desired_jpeg_size_kb) < tolerated_file_size_deviation:
                    remove_image(destinationFolder,base_name,'jpeg')
                    break        
                    
                print(f"{base_name} with quality {new_jpeg_quality} overshoot")
                save_jpeg_img(img=image,name=base_name,quality=new_jpeg_quality - jpeg_quality_step)
                jpeg_quality_70k_compression[base_name] = new_jpeg_quality - jpeg_quality_step
                image.close()
                break
            elif new_jpeg_quality >= 90:
                
                if not np.abs(jpeg_size-desired_jpeg_size_kb) < tolerated_file_size_deviation:
                    remove_image(destinationFolder,base_name,'jpeg')
                    break
                    
                jpeg_quality_70k_compression[base_name] = new_jpeg_quality
                image.close()
                print(f"{base_name} used highest quality !!!")
                break
                
            file_size_one_step_before = jpeg_size
            
        elif jpeg_size > desired_jpeg_size_kb:
            """we should decrease jpeg quality to reach desired file size on disk. """
            new_jpeg_quality -= jpeg_quality_step
            save_jpeg_img(img=image,name=base_name,quality=new_jpeg_quality)
            jpeg_size = get_image_size(destinationFolder,base_name,'jpeg')

            if np.abs(file_size_one_step_before - desired_jpeg_size_kb) <= np.abs(jpeg_size-desired_jpeg_size_kb):
                
                if not np.abs(jpeg_size-desired_jpeg_size_kb) < tolerated_file_size_deviation:
                    remove_image(destinationFolder,base_name,'jpeg')
                    break
                    
                print(f"{base_name} with quality {new_jpeg_quality} overshoot")
                save_jpeg_img(img=image,name=base_name,quality=new_jpeg_quality + jpeg_quality_step)
                jpeg_quality_70k_compression[base_name] = new_jpeg_quality + jpeg_quality_step
                image.close()
                break
            elif new_jpeg_quality <= 5:
                
                if not np.abs(jpeg_size-desired_jpeg_size_kb) < tolerated_file_size_deviation:
                    remove_image(destinationFolder,base_name,'jpeg')
                    break       
                    
                jpeg_quality_70k_compression[base_name] = new_jpeg_quality
                image.close()
                print(f"{base_name} used lowest quality !!!")
                break    
                 
            file_size_one_step_before = jpeg_size
        

4 was below threshold
6 was below threshold
8 was below threshold
17 was below threshold
18 was below threshold
20 was below threshold
21 was below threshold
23 was below threshold
24 was below threshold
25 was below threshold
27 was below threshold
28 was below threshold
31 was below threshold
32 was below threshold
33 was below threshold
36 was below threshold
37 was below threshold
39 was below threshold
40 was below threshold
41 was below threshold
47 was below threshold
48 was below threshold
50 was below threshold
58 was below threshold
59 was below threshold
60 was below threshold
62 was below threshold
64 was below threshold
67 was below threshold
68 was below threshold
71 was below threshold
72 was below threshold
74 was below threshold
77 was below threshold
79 was below threshold
81 was below threshold
84 was below threshold
87 was below threshold
89 was below threshold
90 was below threshold
91 was below threshold
94 was below threshold
95 was below threshold
96 was below t

In [84]:
jpeg_quality_70k_compression.values()

dict_values([60, 24, 28, 26, 20, 58, 20, 72, 60, 24, 32, 52, 24, 34, 32, 66, 28, 28, 66, 42, 30, 60, 56, 42, 28, 60, 42, 42, 50, 30, 26, 36, 26, 34, 22, 22, 28, 48, 32, 34, 66, 20, 56, 28, 28, 28, 62, 32, 62, 52, 22, 50, 54, 38, 26, 64, 26, 28, 24, 46, 48, 56, 52, 22, 42, 32, 40, 20, 70, 44, 60, 24, 26, 28, 22, 48, 62, 54, 30, 24, 76, 28, 44, 30, 22, 22, 56, 60, 24, 18, 60, 72, 32, 26, 24, 22, 36, 20, 24, 24, 60, 70, 32, 30, 34, 32, 34, 34, 26, 68, 34, 48, 54, 28, 22, 26, 30, 34, 22, 34, 28, 20, 28, 22, 24, 66, 32, 64, 44, 22, 30, 24, 22, 50, 40, 26, 48, 22, 50, 32, 50, 34, 50, 30, 26, 26, 22, 22, 24, 20, 22, 78, 26, 48, 22, 46, 24, 30, 34, 28, 28, 30, 30, 30, 22, 24, 60, 22, 20, 30, 24, 22, 20, 28, 54, 32, 38, 24, 46, 28, 62, 30, 26, 38, 20, 56, 40, 28, 26, 54, 24, 38, 22, 20, 70, 26, 36, 26, 54, 60, 26, 60, 56, 46, 32, 32, 50, 56, 34, 30, 44, 26, 22, 26, 50, 38, 28, 36, 44, 42, 60, 24, 24, 60, 26, 46, 30, 46, 28, 20, 26, 62, 46, 40, 56, 32, 24, 72, 26, 54, 44, 26, 50, 20, 22, 66, 24,